<a href="https://colab.research.google.com/github/Sambhavmahajan11/swym_project/blob/main/swym.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


** Analyzing Product Reviews **
---



In [ ]:
!gzip -d "/content/drive/MyDrive/SWYM assignment/AMAZON_FASHION.json.gz"

In [ ]:
!gzip -d "/content/drive/MyDrive/SWYM assignment/meta_AMAZON_FASHION.json.gz"

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#importing the dataset
import pandas as pd
review_df = pd.read_json('/content/drive/MyDrive/SWYM assignment/AMAZON_FASHION.json', lines=True)
product_df=pd.read_json('/content/drive/MyDrive/SWYM assignment/meta_AMAZON_FASHION.json',lines=True)

In [3]:
print(review_df.shape)
print(product_df.shape)

(883636, 12)
(186637, 15)


In [4]:
review_df.head()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,5,True,"10 20, 2014",A1D4G1SNUZWQOT,7106116521,Tracy,Exactly what I needed.,perfect replacements!!,1413763200,NaN,NaN,NaN
1,2,True,"09 28, 2014",A3DDWDH9PX2YX2,7106116521,Sonja Lau,"I agree with the other review, the opening is ...","I agree with the other review, the opening is ...",1411862400,3.0,NaN,NaN
2,4,False,"08 25, 2014",A2MWC41EW7XL15,7106116521,Kathleen,Love these... I am going to order another pack...,My New 'Friends' !!,1408924800,NaN,NaN,NaN
3,2,True,"08 24, 2014",A2UH2QQ275NV45,7106116521,Jodi Stoner,too tiny an opening,Two Stars,1408838400,NaN,NaN,NaN
4,3,False,"07 27, 2014",A89F3LQADZBS5,7106116521,Alexander D.,Okay,Three Stars,1406419200,NaN,NaN,NaN


In [5]:
#Removing the unimportant columns
review_df=review_df.drop(['reviewerName','unixReviewTime','style','image','reviewTime'], axis=1)

In [6]:
review_df['text'] = review_df['summary'] +" "+ review_df['reviewText']
# # Dropping the original columns
review_df = review_df.drop(['summary', 'reviewText'], axis=1)

In [7]:
review_df.columns

Index(['overall', 'verified', 'reviewerID', 'asin', 'vote', 'text'], dtype='object')

In [8]:
review_df['vote'] = review_df['vote'].fillna(0)

In [9]:
df = review_df.dropna()

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 881900 entries, 0 to 883635
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   overall     881900 non-null  int64  
 1   verified    881900 non-null  bool   
 2   reviewerID  881900 non-null  object 
 3   asin        881900 non-null  object 
 4   vote        881900 non-null  float64
 5   text        881900 non-null  object 
dtypes: bool(1), float64(1), int64(1), object(3)
memory usage: 41.2+ MB


In [11]:
!pip install vaderSentiment

In [12]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sentiment = SentimentIntensityAnalyzer()
# nlp = sentiment.polarity_scores(df['text'][1])


In [13]:
l = []
for i in range(len(df)):
  try:
    nlp = sentiment.polarity_scores(df['text'][i])
  except:
    nlp = {'pos': 0.0, 'neg': 0.0}
  l.append(nlp['pos'] - nlp['neg'])
df['senti_score'] = l
print(review_df.head())

   overall  verified      reviewerID        asin  vote  \
0        5      True  A1D4G1SNUZWQOT  7106116521   0.0   
1        2      True  A3DDWDH9PX2YX2  7106116521   3.0   
2        4     False  A2MWC41EW7XL15  7106116521   0.0   
3        2      True  A2UH2QQ275NV45  7106116521   0.0   
4        3     False   A89F3LQADZBS5  7106116521   0.0   

                                                text  
0      perfect replacements!! Exactly what I needed.  
1  I agree with the other review, the opening is ...  
2  My New 'Friends' !! Love these... I am going t...  
3                      Two Stars too tiny an opening  
4                                   Three Stars Okay  


<ipython-input-13-5b8c488c9ed6>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['senti_score'] = l


In [19]:
import pickle
with open("/content/drive/MyDrive/SWYM assignment/file.csv",'wb') as f:
  pickle.dump(df,f)

In [15]:
import pickle
with open("/content/drive/MyDrive/SWYM assignment/file.csv",'rb') as f:
  df = pickle.load(f)

In [20]:
df['overall'] = df['overall'].replace(3, 0.1)
df['overall'] = df['overall'].replace(4, 0.3)
df['overall'] = df['overall'].replace(2, 0.3)
df['overall'] = df['overall'].replace(1, 0.6)
df['overall'] = df['overall'].replace(5, 0.6)

In [21]:
df.head()

,overall,verified,reviewerID,asin,vote,text,senti_score
0,0.6,True,A1D4G1SNUZWQOT,7106116521,0.000000,perfect replacements!! Exactly what I needed.,0.461
1,0.3,True,A3DDWDH9PX2YX2,7106116521,0.003106,"I agree with the other review, the opening is ...",0.061
2,0.3,False,A2MWC41EW7XL15,7106116521,0.000000,My New 'Friends' !! Love these... I am going t...,0.183
3,0.3,True,A2UH2QQ275NV45,7106116521,0.000000,Two Stars too tiny an opening,0.000
4,0.1,False,A89F3LQADZBS5,7106116521,0.000000,Three Stars Okay,0.487


In [22]:
max_value = df['vote'].max()
min_value = df['vote'].min()

df['vote'] = (df['vote'] - min_value) / (max_value - min_value)


In [31]:
df['score']=df['overall']*df['senti_score']
# Apply the vectorized operation
df['score'] = df['score'] * (0.75 * df['verified'] + 0.25 * ~df['verified'])

In [33]:
df.head()

,overall,verified,reviewerID,asin,vote,text,senti_score,score
0,0.6,True,A1D4G1SNUZWQOT,7106116521,0.000000,perfect replacements!! Exactly what I needed.,0.461,0.207450
1,0.3,True,A3DDWDH9PX2YX2,7106116521,0.003106,"I agree with the other review, the opening is ...",0.061,0.013725
2,0.3,False,A2MWC41EW7XL15,7106116521,0.000000,My New 'Friends' !! Love these... I am going t...,0.183,0.013725
3,0.3,True,A2UH2QQ275NV45,7106116521,0.000000,Two Stars too tiny an opening,0.000,0.000000
4,0.1,False,A89F3LQADZBS5,7106116521,0.000000,Three Stars Okay,0.487,0.012175


In [34]:
grouped = df.groupby('reviewerID')['score'].sum()

In [35]:
grouped

reviewerID
A0007604Q2582KFW7N4B    0.060750
A0010606488RW7ZH6EP7    0.000000
A001152055E9KVRHH96L    0.254250
A00181966XZUX7KEGYLH    0.000000
A00205926S885DTOYMX6    0.156150
                          ...   
AZZYFFDBSSBJC          -0.029775
AZZYVTX23JWKM           0.126450
AZZYVURSF9KK6          -0.114300
AZZZMCJO078D2           0.078525
AZZZU2YUCMUUW           0.134550
Name: score, Length: 747844, dtype: float64

In [36]:
mx=grouped.max()
x=grouped[grouped==mx].index.tolist()

In [37]:
review_df[review_df['reviewerID']==x[0]]

,overall,verified,reviewerID,asin,vote,text
70423,5,True,A1RRX286ZRI830,B00515QO4K,0.0,Five Stars cool! u get two straps actually!
90888,5,True,A1RRX286ZRI830,B008YOQ742,4.0,stunning This is a piece of art! everybody who...
115051,5,True,A1RRX286ZRI830,B00CHNXP0K,0.0,I love this bracelet and everybody around me t...
134355,5,True,A1RRX286ZRI830,B00E9LR1AS,0.0,Five Stars Unique and looks great!
180249,3,True,A1RRX286ZRI830,B00JE49KPO,0.0,Nice looking but! It looks nice but the smell....
204028,5,True,A1RRX286ZRI830,B00LVYEIPW,0.0,Five Stars Great vest order two sizes up! they...
210548,5,True,A1RRX286ZRI830,B00MWWS6VE,0.0,Five Stars Lost my case so needed a new one. L...
228673,5,True,A1RRX286ZRI830,B00PFNKKFM,11.0,amazing! great looking statement necklace for ...
232257,5,True,A1RRX286ZRI830,B00PV9XNGI,0.0,Five Stars very nice little addition to any Su...
308301,4,True,A1RRX286ZRI830,B013Y6OON0,0.0,Not what I imagined but still pretty cool. You...


In [38]:
x

['A1RRX286ZRI830']

In [39]:
print(mx)

3.969675


In [40]:
from collections import defaultdict

dp = defaultdict(int)

for index, row in df.iterrows():
    if row['score'] >0:
      dp[row['asin']]+=1

In [41]:
# Find the maximum length of values
max_length = max(dp.values())

# Find the keys with the maximum length of values
keys_with_max_length = [key for key, value in dp.items() if value== max_length]

# Print the keys with maximum length of values
print("Key(s) with maximum length of values:", keys_with_max_length)

Key(s) with maximum length of values: ['B000KPIHQ4']


In [42]:
max_length

3656